<a href="https://colab.research.google.com/github/FastalGroup/CorsoLangChain/blob/main/AgenteRAG_CorsoBase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Realizzazione di un Agente ChatBot RAG

In questo notebook viene descritta la costruzione di un **Agente Intelligente Conversazionale** che è in grado di assistere un utente relativamente ad uno specifico dominio applicativo.

Il ChatBot sfrutta le capacità di comprensione del linguaggio naturale di un Modello di IA, **Large Language Model**, di ultima generazione per comprendere le richieste dell'utente, reperire le informazioni necessarie da un database che contiene documenti specifici sul dominio applicativo (**base di conoscenza**), e rispondere in modo appropriato all'utente.



## Concetti generali

L'assistente descritto e realizzato in questo notebook si avvale di strumenti e tecnologie che, al momento, costituiscono lo stato dell'arte nel settore dell'**Intelligenza Artificiale Generativa**.

Nel seguito sono descritti i componenti e i paradigmi utilizzati nell'architettura del ChatBot.



### LLM - Large Language Model

Il LLM è il componente centrale dell'architettura del nostro assistente.

Un LLM è una **rete neurale** di grandi dimensioni, comprendente diverse centinaia di miliardi di interconnessioni tra nodi di elaborazione (neuroni), strutturate secondo un "*modello*" derivato dalla interconnessione di strutture chiamate "*transformer*".

I primi **LLM** presentati dai grandi provider come **OpenAI** e **Google** erano sistemi in grado di generare risposte in linguaggio naturale, corrette dal punto di vista grammaticale, sintattico e semantico, in risposta ad un testo ricevuto come input.

A partire dal 2022, i modelli presentati e offerti come servizio sul mercato, hanno una struttura e una funzionalità molto più complessa, orientata ad interazioni di tipo conversazionale.

Questi nuovi modelli sono solitamente indicati come **Chat LLM** per distinguerli dai più semplici e primitivi LLM.

Questi nuovi modelli si contraddistinguono per alcune capacità funzionali:
- sono **multimodali**, nel senso che possono ricevere in input e generare in output contenuti non limitati al solo testo. In particolare gestiscono nativamente immagini, video e suoni. (Nel nostro caso non abbiamo bisogno di utilizzare questa capacità).
- prevedono l'impiego di "**tool**"
- prevedono **messaggi** di input associati a **ruoli** specifici, ad esempio istruzioni di sistema, input utente, messaggi dell'assistente
- hanno capacità di ragionamento avanzate e sono in grado di fornire istruzioni

Ad oggi, questi modelli sono disponibili esclusivamente in modalità ***cloud computing***, accessibili mediante API REST.

Per il nostro prototipo abbiamo scelto di utilizzare come modello: **Claude 3.5 v.2 di Anthropic**.

Il software è tuttavia facilmente adattabile a qualunque altro modello di pari capacità.




### Paradigma RAG - Retrieval Augmented Generation

I **LLM** di ultima generazione mostrano capacità di comprensione e conoscenze in campi molto diversi tra loro, tanto da apparire quasi onniscienti.

Il modello utilizzato in questo prototipo, ad esempio, è sicuramente in grado di rispondere in modo corretto a domande che riguardano argomenti molto specifici, relativamente ai quali esiste ampia documentazione di pubblico dominio, accessibile via Internet.

I principali LLM di mercato incorporano queste conoscenze di pubblico dominio aggiornate alla data di addestramento del modello e sono oramai in grado di fornire indicazioni corrette con un elevato grado di probabilità.

Questi LLM ovviamente non sono in grado di conoscere informazioni più recenti rispetto alla data del loro ultimo aggiornamento e informazioni specifiche di una azienda che non sono pubblicamente disponibili e accessibili.

Pertanto, la realizzazione di un assistente basato su un modello pre-addestrato **SOA** (*State-Of-the-Art*), che sia in grado di offrire un servizio personalizzato per una particolare applicazione, richiede una programmazione specifica.

Una prima soluzione potrebbe essere quella di effettuare un supplemento di addestramento (detto ***fine tuning***) del modello, per ottenere una versione "personalizzata" in grado di gestire la conoscenza specifica sul dominio applicativo.

I servizi di *fine tuning* sono offerti da tutti i provider di LLM e la soluzione è tecnicamente praticabile. Tuttavia presenta due svantaggi:

- Il **costo** del fine tuning e il successivo **costo** di accesso alla versione "personalizzata" del LLM sono molto **elevati**.
- In caso di **aggiornamenti della documentazione** che fa parte della base di conoscenza specifica occorre procedere ad una **nuova**, costosa, **fase di fine tuning**.

La seconda soluzione, in genere molto meno onerosa e più adatta al nostro contesto, sfrutta una caratteristica dei moderni **Chat LLM** che consiste nella possibilità di fornire assieme alla domanda dell'utente, una serie di **istruzioni** e **informazioni di contesto**.

L'**input** ad un **Chat LLM** è un dato strutturato, solitamente in formato JSON, che contiene diversi campi detti ***messages***, ciascuno dei quali, oltre alla stringa di testo che è il contenuto vero e proprio del messaggio, contiene anche meta informazioni come ad esempio il **ruolo** dell'attore del messaggio (cioè a che titolo viene fornito il messaggio).

Questa struttura è quella che propriamente viene chiamata **prompt**.

Un prompt preparato per un moderno Chat LLM contiene solitamente diversi messaggi, come ad esempio:

- Istruzioni operative che spiegano al LLM come comportarsi nell'elaborare la risposta, indicando eventuali vincoli o modalità di strutturazione della risposta. In genere questo tipo di messaggio ha un ruolo indicato come "**system**".
- Contesto di riferimento, cioè un testo che contiene o esplicita informazioni aggiuntive che potrebbero non essere note al LLM oppure non essere quelle immediatamente utilizzate per elaborare la risposta. Un messaggio di questo tipo potrebbe avere un ruolo "**system**" ma alcuni LLM prevedono un ruolo specifico chiamato "**assistant**".
- Messaggi che costituiscono la **storia precedente della conversazione**, ciascuno con i vari ruoli. Questo è importante per poter gestire correttamente i cosiddetti thread di conversazione. Le chiamate alle API degli LLM sono solitamente stateless.
- Domanda dell'utente, in genere indicata con il ruolo "**user**".

La struttura del prompt è specifica per ogni modello LLM.

I **Chat LLM** sono specificamente addestrati per tenere in debita considerazione i messaggi e soprattutto i ruoli con i quali vengono forniti.

Il paradigma **RAG** sfrutta questa caratteristica.
In prima battuta potremmo pensare di inserire il testo di tutta la documentazione specifica come messaggio di contesto, in modo che il LLM risponda tenendo in considerazione queste informazioni.

In realtà questo è possibile solo se tali informazioni, valutate in termini di parole (più esattamente *token*), hanno una dimensione compatibile con il LLM.
Un LLM come quello che abbiamo utilizzato ha una "finestra di contesto" di circa 200.000 token. Si tratta di una buona capacità, ma sufficiente a gestire non più di due o tre manuali.
Inoltre il costo del servizio di un Chat LLM è calcolato in base al numero di token forniti in input e generati in output.

Sarebbe quindi più opportuno passare come "contesto" solo i documenti, o i paragrafi, pertinenti alla query dell'utente.

È su questo principio che viene definito un tipico **processo RAG**:

1. l'utente inserisce la sua **query**;
2. il sistema utilizza la query per selezionare i **documenti (o paragrafi) pertinenti** dalla "*base di conoscenza*";
3. il sistema prepara un **prompt** opportuno, composto da **istruzioni**, sequenza di **testi recuperati** al passo precedente e **query** dell'utente;
4. il sistema **invoca** il LLM fornendo il **prompt** preparato al passo precedente;
5. il LLM genera la sua **risposta** che sarà fortemente influenzata dal contesto e dalle istruzioni fornite.

Con la tecnica **RAG** è possibile costruire ChatBot personalizzati che forniscono risposte corrette su specifici domini di conoscenza.


### Embeddings

Nel paragrafo precedente è stato descritto un tipico processo **RAG**.

Al **punto 2** è stato indicato che la **query** dell'utente viene utilizzata per **selezionare** dalla base di conoscenza i **documenti o i paragrafi pertinenti**.

Si tratta dell'aspetto più critico in tutto il processo.

Come nel nostro caso, molto spesso, la base di conoscenza è costituita da un insieme di documenti di varia origine e natura. **Come si individuano le parti di testo pertinenti?**

Occorre avere una strutturazione della base di conoscenza che sia un **database** su cui possono essere effettuate **ricerche di tipo semantico**.

È su questo aspetto che interviene la tecnologia degli **embeddings**.

Si tratta di uno dei prodotti più importanti della ricerca sull'elaborazione del linguaggio naturale (**NLP**) che ha portato ai moderni LLM.

La tecnica degli **embeddings** è una tecnica che permette di associare ad un testo, più o meno lungo, (frase, paragrafo o intero documento) un **vettore di numeri** di tipo **`float`** di lunghezza fissa.

Gli algoritmi di **embeddings** sono a loro volta delle reti neurali addestrate su corpus di testo molto ampi.

L'addestramento su enormi quantità di testi di questi algoritmi è una delle fasi propedeutiche allo sviluppo di un LLM.

Un **modello di embeddings** è una **rete neurale**, solitamente basata su *transformers* che è stata addestrata a riconoscere le relazioni sintattiche e semantiche tra le parole (in realtà tra token) che compongono un testo.

Il **vettore risultante** (*vettore di embeddings*) è quindi un insieme di numeri che rappresentano in qualche modo aspetti sintattici, ma soprattitto **semantici**, del testo di origine.

La dimensione dei **vettori di embeddings** utilizzati nei moderni sistemi di IA oscilla tra 256 e 3072.

Se consideriamo un **vettore di embeddings** come un punto in uno spazio n-dimensionale possiamo associare alle distanze geometriche tra punti (vettori) diversi la relazione semantica tra i corrispondenti testi di origine.

Gli algoritmi di **embeddings** rendono possibile trattare le relazioni semantiche tra testi utilizzando formule matematico-geometriche.

Gli **embeddings** sono la chiave per realizzare database con possibilità di ricera semantica.

Nel nostro caso, abbiamo scelto di utilizzare come algoritmo di embeddings il recente modello **Text Embeddings 3 Large** di **OpenAI** che genera vettori di dimensione **3072**.




### Database vettoriale

Il **Database Vettoriale** è uno dei componenti chiave di una soluzione **RAG**.

Si tratta di uno specifico DBMS ottimizzato per la memorizzazione di **vettori** e la relativa indicizzazione.

In particolare, per implementare una soluzione **RAG** occorre un **database vettoriale** che sia indicizzato, cioè che associ i vettori memorizzati (che sono degli **embeddings**) con i relativi file di origine.

Il **database vettoriale** o **vectorstore** costituisce la base di conoscenza su cui il processo **RAG** effettua la **ricerca semantica**.

La sua costruzione e il suo popolamento deve tenere in considerazione le esigenze dimensionali sia dei LLM, sia degli algoritmi di embeddings.

I **modelli di embeddings** hanno dei limiti alla lunghezza del testo che può essere fornito in input, così come i **LLM** hanno dei limiti sulla lunghezza complessiva del **prompt**.

La costruzione del **vectorstore** avviene pertanto secondo il seguente processo:

1. I documenti di origine vengono suddivisi in frammenti, chiamati **chunk**, di dimensioni adeguate (solitamente di alcune migliaia di parole o token);
2. ciascun **chunk** viene fornito in input al modello di **embeddings** per ottenere il corrispondente vettore;
3. ciascun **vettore** viene quindi aggiunto nel database vettoriale;
4. il **database vettoriale** viene indicizzato correlando ogni **vettore** al suo **chunk** di testo. I **chunk** componenti un singolo documento sono correlati al documento di origine.

Il **vectorsore** deve offrire **funzionalità di ricerca di tipo semantico**.

Nel prototipo costruito in questo notebook abbiamo usato come DBMS vettoriale il prodotto *open source* **Chroma**.

Si tratta di un prodotto molto leggero, serializzato su file system, specializzato per applicazioni di IA.

In un contesto di produzione si può utilizzare un qualunque DBMS vettoriale di classe adeguata.

L'offerta AWS è abbastanza ampia in questo settore.

### Architettura a catena o "chain"

La struttura di massima descritta nel paragrafo dedicato al paradigma **RAG** rappresenta lo schema più semplice e controllabile per un assistente personalizzato.

Usando questo tipo di architettura, l'applicazione viene realizzata programmando i singoli blocchi della sequenza.

Il **flusso di esecuzione** segue un percorso lineare, detto *chain*.

Ogni chiamata del ChatBot attiva un thread di esecuzione composto dalla sequenza lineare di operazioni: **`retrieval -> costruzione prompt -> invocazione del LLM`**.

Questa soluzione fornisce allo sviluppatore il pieno controllo del flusso di elaborazione, ma in alcuni casi presenta una serie di svantaggi che vedremo più avanti, con degli esempi.

### Struttura conversazionale e Chat History

Lo schema di ChatBot descritto finora è di tipo ***stateless***.

Ogni invocazione dell'assistente avvia un thread di elaborazione che non tiene conto di eventuali invocazioni precedenti effettuate dallo stesso utente.

In altre parole, il ChatBot gestisce la query dell'utente in modo puntuale: botta e risposta, senza memoria dei messaggi scambiati in precedenza.

Nella pratica, i chat bot a cui siamo abituati, ad esempio ChatGPT, permettono un'interazione in cui il contesto è significativo.

Eventuali domande poste successivamente alla prima, possono implicitamente fare riferimento ai messaggi precedenti.

Un frammento di conversazione reale, tratta da una applicazione che assiste gli utenti di un prodotto di e-procurement, potrebbe essere il seguente:

- Utente: "Il sistema gestisce le gare?"
- ChatBot: "Si, il prodotto.... fornisce supporto, bla, bla, bla..."
- Utente: "Come ne inserisco una?"
- ...

È evidente che con una architettura di ChatBot stateless, la gestione della seconda domanda sarebbe imprecisa. La frase "*Come ne inserisco una?*" assume un senso preciso solo facendo riferimento ai messaggi precedentemente scambiati.

Se si vuole realizzare un assistente in grado di gestire una **conversazione**, cioè un *thread* di domande e risposte, occorre gestire due nuovi elementi:

1. la definizione di "**sessioni**" utente;
2. il mantenimento della **storia** di tutti i messaggi scambiati durante la sessione.

Le API offerte dai provider degli LLM sono di tipo *stateless*. Ad ogni nuova chiamata viene avviato sul cloud del provider un nuovo thread di esecuzione. Non viene offerto supporto per la gestione di sessioni. Ogni chiamata è una nuova sessione che si apre con l'invocazione del LLM e si chiude con la ricezione della risposta.

La gestione delle sessioni e il mantenimento della cosiddetta **chat history** è a cura del software dell'assistente.



### Tool

I LLM SOA prevedono l'esistenza di **tool**.

Un **tool** è una funzione software in grado di elaboare informazioni, eseguire un compito, generare delle nuove informazioni.

La disponibilità di una o più funzioni di questo tipo, è un parametro che può essere passato ad un LLM attraverso il **prompt**.

Quello che riceve in input il LLM è un elenco di nomi di **tool** (sono identificatori definiti dallo sviluppatore) associati ad una descrizione in linguaggio naturale degli scopi del **tool**, della sua funzione e delle sue capacità.

Se nel **prompt** viene fornita la **lista dei tool** il LLM può decidere di non rispondere immediatamente alla **query** dell'utente, ma di fornire una risposta (le risposte sono in genere strutture JSON) in cui è indicata l'esigenza di invocare uno o più **tool** con determinati parametri di input.

È importante osservare che il LLM **non è in grado di eseguire o invocare direttamente le funzioni (tool)**, ma si limita a fornire come risposta una richiesta di invocazione dei **tool** appropriati.

Il programma può essere strutturato per eseguire in modo automatico le chiamate alle relative funzioni, oppure può essere strutturato per demandare la decisione di eseguire una certa funzione all'utente. Quest'ultimo tipo di flusso viene definito ***man in the middle***.

In ogni caso il controllo sull'esecuzione dei **tool** è demandato al software dell'assistente.

Dopo aver chiamato le funzioni richieste, i risultati vengono passati al LLM con un nuovo **prompt**.

Il LLM può chiedere ulteriori esecuzioni di **tool** finché la sua logica di elaborazione continua a fornire questo tipo di risultato, oppure fornire la risposta alla **query** e terminare il processo.

Si tenga sempre a mente che la risposta, e quindi la logica di ragionamento, del LLM è influenzato anche dalle eventuali **istruzioni** fornite nel **prompt** a corredo della query utente e del contesto.

### Agenti e applicazioni agentiche

La capacità di gestire i **tool**, da parte di un LLM, rende possibile strutturare l'applicazione in un modo diverso.

Nella struttura a **chain** ogni blocco della catena può anche essere visto come una specifica ***funzione*** nel senso di modulo software realizzato con il linguaggio di programmazione.

I singoli **blocchi**, cioè le singole **funzioni**, sono eseguite in un ordine prestabilito, o **secondo un flusso predeterminato dalla logica di controllo del programma** (tramite le classiche istruzioni `if..then..else` e `loop`).

Se, al contrario, implementiamo ogni blocco come una effettiva funzione programmatica che rispetta la struttura di *input* ed *output* previste dalle API del LLM per la definizione dei **tool**, possiamo **demandare il controllo del flusso effettivo di elaborazione al LLM**.

In altre parole, dopo aver fornito il **prompt** con la **query**, le **istruzioni**, il **contesto**, la **chat history** e la **lista dei tool**, sarà il LLM, tramite le sue risposte, a determinare le operazioni da eseguire, cioè le chiamate ai singoli **tool**, e quindi in ultima analisi a detrminare una certa sequenza di operazioni, per rispondere alla **query** posta dall'utente.

Una struttura di questo tipo è chiamata **agente**.

Più in generale, le applicazioni in cui il flusso di elaborazione è totalmente o parzialmente controllato tramite le risposte di un LLM sono definite **applicazioni agentiche**.

Si tratta di sistemi molto versatili e potenti, ma meno controllabili rispetto alle **chain**.

## Realizzazione di un ChatBot personalizzato con tecnica RAG

In questo Notebook sono esplorate e realizzate tutte le architetture e tutti i componenti descritti nella premessa.

L'obiettivo è quello di mostrare soluzioni alternative e un toolkit di strumenti software per speriemntare, testare e validare le diverse architetture e i singoli componenti.

Lo sviluppo del software è articolato in sezioni:

1. Nella prima sezione viene mostrato come costruire un **vectorstore** e come popolare e indicizzare il **database vettoriale** effettuando il **chunking** e l'**embeddings** dei documenti che costituiscono la **base di conoscenza**. Vengono anche predisposte le istruzioni di utility per salvare e ripristinare il **database vettoriale** rendendolo persistente rispetto all'esecuzione del presente notebook.

2. Nella seconda sezione viene costruito un **ChatBot RAG** di tipo **stateless** con architettura a **chain**.

3. Nella terza sezione viene costruito un **ChatBot RAG** di tipo **conversazionale**, in grado di gestire **sessioni utente** e mantenere una **chat history**.

4. Nella quarta sezione viene costruito un **ChatBot RAG** di tipo **agentico**.

## Sviluppo del software

### Linguaggio di programmazione

Per lo sviluppo dell'assistente è stato utilizzato il linguaggio di programmazione **Python**.

La scelta del **Python** è oggi obbligata da alcune importanti ragioni di opportunità:

- Il Python è, allo stato dell'arte, il linguagio d'elezione dell'Intelligenza Artificiale. La disponibilità di strumenti e librerie in Python supera di gran lunga l'offerta di strumenti comparabili, disponibili in linguaggi come Java.
- Il Python ha una sintassi più intuitiva, pulita e facilmente comprensibile anche a chi non conosce il linguaggio. Si presta pertanto ad essere uno strumento ideale per il trasferimento di know how.
- Il Python è interpretato e si presta particolarmente bene ad essere utilizzato in modo interattivo e incrementale, rendemndo possibile strumenti come il presente notebook.
- Tramite il Python è possibile abbattere i tempi di realizzazione di sistemi di IA come quello qui presentato.



### Framework di astrazione

Uno dei problemi nello sviluppo di applicazioni basate sulle tecnologie LLM più recenti è costituito dall'enorme fluidità del settore che è in continua e rapida evoluzione.

Le prestazioni dell'applicazione potrebbero migliorare notevolmente nell'immediato futuro avvalendosi dei modelli di **embeddings** e **LLM** più recenti.

Inoltre, i modelli scelti per l'implementazione, che oggi costituiscono lo stato dell'arte, potrebbero rapidamente risultare obsoleti.

Questa situazione non costituisce un problema per chi si occupoa di ricerca e sviluppo, trattandosi di un contesto dove le variazioni di scenario e le innovazioni tecnologiche sono il contesto quotidiano, mentre può essere una importante criticità per un contesto di produzione.

Ipotizzando come contesto di produzione un *deployment* su una infrastruttura cloud come **AWS** o **Microsoft Azure**, è facile constatare che l'offerta di servizi e modelli di IA, disponibili direttamente su queste piattaforme, è in costante evoluzione. La stessa documentazione ufficiale dei cloud provider fatica a stare dietro agli aggiornamenti.

Ogni nuovo modello, sia di **embeddings** che **Chat LLM** porta con se le sue API e le sue specificità.

I diversi modelli offerti dai provider leader di mercato sono simili nella logica astratta ma diversi nelle specifiche connesse alle strutture dati e ai messaggi in input e output.

Una applicazione come quella qui presentata gestisce elementi importanti e complessi, quali **prompt**, **messaggi**, definizioni di **tool**, **query semantiche** e **chat history**.

Una volta scelta e adottata una combinazioni di modelli, e relative API, il passaggio a servizi diversi comporta inevitabilmente un *refactoring* importante del codice sorgente.

Per questo motivo è stata effettuata una scelta, molto comune nei contesti destinati allo sviluppo di applicazioni che andranno in produzione, di utilizzare una piattaforma di astrazione, specifica per la realizzazione di architetture a **chain** e **applicazioni agentiche** mediante **LLM SOA**.

La scelta è inevitabilmente ricaduta sul prodotto, attualmente leader di mercato, denominato **LangChain**.

#### LangChain

**LangChain** offre allo sviluppatore un modello astratto per ciascuna delle entità che abbiamo descritto nella sezione generale.

Attraverso API molto potenti, e ben documentate, è possibile realizzare con metodi dichiarativi **chain** di funzionalità tipiche delle applicazioni **RAG**, **vectorstore**, **chat history** e gestire **sessioni utente**.

L'interfaccia delle API è **indipendente dai servizi effettivamente utilizzati**.

**LangChain** offre l'integrazione delle sue API con la quasi totalità dei provider operanti sul mercato, ed è la soluzione raccomandata da tutti i leader di mercato, come **Amazon AWS**, **Microsoft Azure**, **OpenAI** e **Anthropic**.

La migrazione verso soluzioni diverse è una operazione semplice e immediata, che richiede modifiche minime e puntuali al codice sorgente.

La programmazione tramite **LangChain** viene svolta ad un livello di astrazione molto elevato e lo sviluppo di applicazioni robuste e solide dal punto di vista architetturale è estremamente veloce.


# Sviluppo del software

## Installazione dei package Python necessari.

L'istruzione contenuta nella cella seguente, è una istruzione Linux che installa tramite il Package Manager PIP, i moduli che useremo nel software:

- **langchain** è il package principale delle librerie LangChain
- **langchain_community** contiene le librerie definite dalla community di utenti di Lang Chain, da cui estrarremo alcune classi per il nostro software
- **langchain-chroma** è il package che contiene l'interfaccia di integrazione verso il DB vettoriale **Chroma**
- **pypdf** è una libreria molto usata in Python per la lettura dei file in formato PDF
- **langchain-openai** è il package che contiene l'interfaccia di integrazione verso i servizi **OpenAI**
- **langchain-anthropic** è il package che contiene l'interfaccia di integrazione verso i servizi **Anthropic**
- **langgraph** è un package della suite LangChain che contiene le classi e le API per la definizione di applicazioni agentiche

In [ ]:
!pip install -qU langchain langchain_community langchain-chroma pypdf langchain-openai langchain-anthropic langgraph

## Accesso ai provider AI

Il nostro software utilizza sia i servizi di OpenAI che di Anthropic.

Per entrambe le piattaforme, l'accesso alle relative API avviene mediante una chiave segreta, il cui valore può essere passato direttamente nei metodi di invocazione delle API oppure impostato in una variabile di ambiente.

Inserire il valore della chiave segreta in modo esplicito nel codice non è una pratica consigliabile, per cui abbiamo adottato il secondo metodo.

Nella cella di codice che segue, vengono impostati i valori delle variabili di ambiente a partire da due valori *secret* di Google Colab.

I *secret* impostati su colab sono specifici dell'account Google dell'utente che esegue il notebook e non vengono memorizzati nel notebook.

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")

## Sezione 1 - Realizzazione del Vectorstore

**Il processo di realizzazione e gestione di un vectorstore per applicazioni RAG avviene off-line rispetto all'esecuzione dell'assistente.**

La piattaforma DBMS utilizzata per creare il **database vettoriale** dell'applicazione, non influisce sulle prestazioni della nostra applicazione in termini di "inferenza".

Tali prestazioni dipendono dal modello utilizzato per l'algoritmo di **embeddings**.

Lo scopo del DB vettoriale è semplicemente quello di memorizzare i **vettori di embeddings** e indicizzarli in riferimento ai file di origine.

In un contesto di produzione, useremo un sistema DBMS Vettoriale in grado di offrire prestazioni adeguate al carico transazionale previsto.

Nel caso di questo notebook, possiamo usare il sistema **Chroma** perfettamente adeguato anche in relazione alla dimensione della Base di Conoscenza fornita.

La cella seguente, contiene il codice di definizione di una **classe manager** che incapsula, in logica *object oriented*, le funzionalità di un **vectorstore** finalizzato alla nostra applicazione **RAG**.

Per la realizzazione è stato utilizzato il **layer di integrazione** tra **LangChain** e **Chroma**.

Il codice è facilmente interpretabile.

Sono definiti i metodi per **inizializzare un nuovo vectorstore**, per **aggiungere**, **modificare** ed **eliminare** un singolo documento PDF e per **aggiungere tutti i file PDF presenti in una determinata directory**.

I documenti caricati sono identificati da **chiavi univoche** generate automaticamente a partire dal nome del file. Un apposito file JSON mantiene la corrispondenza tra i file caricati e i rispettivi ID.

Il **vectorstore** è stato dichiarato **persistente** attraverso la impostazione di una apposita **cartella** (directory) sul file system locale.

La cella contiene esclusivamente la definizione della classe.

Il codice di questa classe è spiegato più avanti.

In [ ]:
import json
import os
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

class VectorStoreManager:
    def __init__(self, persist_directory, embedding_model="text-embedding-3-large"):
        """
        Inizializza il VectorStoreManager.

        :param persist_directory: Directory dove il vectorstore sarà persistente.
        :param embedding_model: ID del modello di embeddings da utilizzare.
        """
        self.persist_directory = persist_directory

        # Inizializza l'embedding di OpenAI
        self.embedding = OpenAIEmbeddings(
            model=embedding_model
        )

        # Assicurati che la directory di persistenza esista
        os.makedirs(self.persist_directory, exist_ok=True)

        # Inizializza o carica il vectorstore
        self.vectorstore = Chroma(
            persist_directory=self.persist_directory,
            embedding_function=self.embedding
        )

        # Carica la mappatura degli ID dei documenti
        self.id_mapping_path = os.path.join(self.persist_directory, "id_mapping.json")
        self.id_mapping = self._load_id_mapping()

    def _load_id_mapping(self):
        if os.path.exists(self.id_mapping_path):
            with open(self.id_mapping_path, 'r') as f:
                return json.load(f)
        return {}

    def _save_id_mapping(self):
        with open(self.id_mapping_path, 'w') as f:
            json.dump(self.id_mapping, f, indent=4)

    def _generate_unique_id(self, file_path):
        return os.path.basename(file_path)

    def load_documents_from_directory(self, directory_path, chunk_size=6000, chunk_overlap=200):
        """
        Carica tutti i documenti PDF da una directory e li aggiunge al vectorstore.

        :param directory_path: Percorso della directory contenente i PDF.
        :param chunk_size: Dimensione dei chunk di testo.
        :param chunk_overlap: Sovrapposizione tra i chunk.
        """
        if not os.path.isdir(directory_path):
            raise ValueError(f"{directory_path} non è una directory valida.")

        for filename in os.listdir(directory_path):
            if filename.lower().endswith('.pdf'):
                file_path = os.path.join(directory_path, filename)
                self.add_document(file_path, chunk_size, chunk_overlap)

        print(f"Tutti i documenti da {directory_path} sono stati caricati nel vectorstore.")

    def add_document(self, file_path, chunk_size=6000, chunk_overlap=200):
        if not os.path.isfile(file_path):
            print(f"Il file {file_path} non esiste. Salto l'aggiunta.")
            return 0, None

        loader = PyPDFLoader(file_path)
        try:
            docs = loader.load()
        except Exception as e:
            print(f"Errore nel caricamento del file {file_path}: {e}")
            return 0, None

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        splits = text_splitter.split_documents(docs)

        id_documento = self._generate_unique_id(file_path)

        for doc in splits:
            doc.metadata['source_id'] = id_documento

        self.vectorstore.add_documents(splits)
        print(f"Aggiunto documento {id_documento} con {len(splits)} chunk.")

        self.id_mapping[id_documento] = file_path
        self._save_id_mapping()

        return len(splits), id_documento

    def remove_document(self, id_documento):
        documents = self.vectorstore.get(where={"source_id": id_documento})

        if not documents:
            print(f"Nessun documento trovato con ID: {id_documento}")
            return

        ids_to_delete = documents['ids']
        self.vectorstore.delete(ids=ids_to_delete)
        print(f"Rimosso documento con ID: {id_documento}")

        if id_documento in self.id_mapping:
            del self.id_mapping[id_documento]
            self._save_id_mapping()

    def get_retriever(self):
        return self.vectorstore.as_retriever()

La classe **`VectorStoreManager`** incorpora in un unico luogo tutte le funzionalità necessarie agli aspetti di **vettorializzazione** della **base di conoscenza** e recupero delle informazioni pertinenti sulla base di ricerche semantiche effettuate secondo il criterio della **similarity**.

La classe è implementat attraverso le librerie specifiche offerte da **LangChain**, che permettono di semplificare lo sviluppo del codice.

Gli oggetti appartenenti a questa classe comprendono un proprio database vettoriale Chroma, reso persistente su file system locale, i metodi per inserire, modificare e rimuovere documenti in formato PDF dal database vettoriale e un metodo (**`get_retriever`**) che consente di definire un oggetto della classe LangChain **`retriever`** che astrae le funzionalità di ricerca semantica del database vettoriale associato all'oggetto.

I parametri passati al costruttore della classe sono:
- il ***path*** della directory (cartella) locale in cui gestire la persistenza
- il modello di **embeddings** da utilizzare per la vettorializzazione dei documenti. Questo parametro ha come default il modello **text-embeddings-3-large**.

Il costruttore inizializza il client Bedrock configurandone l'accesso al modello di **embeddings** passato come parametro.
L'operazione viene gestita dalle API di LangChain di integrazione verso i servizi OpenAI.
La struttura dell'API **`OpenAIEmbeddings`** è analoga alla struttura di tutte le API di integrazione verso qualunque *provider* di modelli IA integrato da **LangChain**. Questo rende immediato e rapido un eventuale, futuro, porting del software verso altri provider.

Successivamente viene **inizializzato** il **database vettoriale** attraverso due sole istruzioni:

- `os.makedirs(self.persist_directory, exist_ok=True)` è un'istruzione di libreria standard Python che verifica l'esistenza della directory (cartella) passata come parametro. Se la cartella non esiste la crea nel path indicato.
- ```
  self.vectorstore = Chroma(
  persist_directory=self.persist_directory,
  embedding_function=self.embedding)
  ```
  Assegna all'attributo **vectorsore** un oggetto LangChain di tipo *vectorstore*, implementato con Chroma, e reso persistente attraverso la cartella di cui al punto precedente. Se la cartella era presesitente, **vectorstore** punta al database esistente nella cartella, se la cartella è stata appena creata, viene creato al suo interno un database vuoto.

Si noti che un oggetto **vectorstore** LangChain è un wrapper che incorpora un normale **database vettoriale** abbinato a funzionalità di **embeddings** offerte dal modello specificato.

Qualunque sia la complessità del DBMS utilizzato, le API di integrazione di LangChain consentono il setup del **vectorstore** secondo questo tipo di schema di codice, astratto e semplice.

L'ultima parte di codice del costruttore, riguarda il completamento della struttura di persistenza, con la gestione di una mappa di identificatori di documento.

Questa parte non è una funzione particolare di LangChain ma è un semplice metodo di indicizzazione che abbiamo deciso di implementare per tenere traccia dei documenti vettorializzati, come sarà chiaro fra poco.

Per comprendere come viene popolato il **vectorstore** possiamo fare riferimento ad uno dei metodi per l'inserimento di un nuovo documento.

Prendiamo come esempio il più semplice: **add_document**

Questo metodo riceve come parametro il path di un documento PDF, che avremo preventivamente salvato sul nostro file system e lo inserisce nel **vectorstore** dopo aver svolto alcuni importanti passaggi:

1. Per prima cosa il documento PDF viene processato da una funzione "**loader**", che fa parte delle API di **LangChain**, che sfrutta la libreria standard **pypdf** per trasformare il file PDF in una lista di stringhe di testo, ciascuna corrispondente ad una pagina del documento PDF.
2. Successivamente ciascuna pagina, oramai in formato txt, viene eventualmente ridotta in frammenti (chunk). La dimensione di questi frammenti è controllata dal parametro **chunk_size**. Questa frammentazione dei testi, viene effettuata in modo da garantire una completezza sintattica e semantica di ciascun frammento. Il parametro **chunk_overlap** determina quanti caratteri o token di sovrapposizione si desiderano tra la fine di un frammento e l'inizio del successivo. Nel nostro caso il default impostato per **chunk_size** è di 6.000 token. Poiché il loader PDF di LangChain ha già suddiviso il documento di partenza in pagine, difficilmente le pagine di un manuale o di una circolare saranno superiori a 6.000 token, per cui è probabile che le pagine non siano ulteriormente suddivise. Il parametro 6.000 è un limite euristico per casi d'uso come il nostro che minimizza i costi delle invocazioni ai modelli LLM senza penalizzare le prestazioni semantiche. L'eventuale ulteriore frammentazione sarebbe comunque demandata alla classe LangChain **RecursiveCharacterTextSplitter** che implementa un algoritmo di frammentazione semantica ben costruito.
3. I singoli frammenti di testo (chunk), che si suppongono sintatticamente e semanticamente completi, vengono quindi sottoposti ad **embeddings**, utilizzando il modello impostato, e aggiunti al **vectorstore**.

Si noti che a questo punto, il vettori contenuti nel **vectorstore** fanno riferimento ai singoli **chunk** e non al documento originale. Un tipico manuale di 100 pagine verrebbe probabilmente ridotto in 100 chunk diversi, uno per ciascuna pagina e nel database vettoriale troveremo 100 vettori, ciascuno dei quali indicizzato e riferito al corrispondente testo.

Nel momento in cui andremo ad inserire ulteriori documenti PDF non sapremmo più a quale documento di origine sono riferiti i singoli vettori.

Questo non è assolutamente un problema per lo schema **RAG** in cui quello che conta è lo specifico frammento di informazione recuperato.

Diventa un problema se vogliamo manutenere la base di conoscenza. Potremmo ad esempio voler eliminare o sostituire un manuale e in questo caso dovremmo effettuare una eliminazione di tutti i vettori associati ad uno specifico documento.

Questo legame può essere gestito con i meta campi del database vettoriale.

Nel nostro caso abbiamo utilizzato un meccanismo piuttosto banale. A partire dal nome del file del documento, generiamo una stringa che costituisce un **ID univoco** per il particolare docuemnto.

Manteniamo traccia del mapping tra gli identificatori e i path d'origine dei documenti in un file in formato JSON che abbiamo aggiunto nella cartella di persistenza del nostro DB.

Quando aggiungiamo un chunk, registriamo nelle meta informazioni l'ID del documento di provenienza.

In questo modo saremo sempre in grado di eliminare dal DB tutti i chunk di un dato documento.

I metodi di aggiornamento del **vectorstore** che abbiamo implementato funzionano su questo principio e dovrebbero essere facilmente comprensibili.




## Formato dei documenti sorgenti della Base di Conoscenza

La **base di conoscenza** fornita da Net4Market è quella tipica di ogni organizzazione.

Le informazioni sono memorizzate su file di vario formato: txt, PDF, Microsoft Office (.doc, .xls, .ppt), Open Office (.odt), ...

Le operazioni di ***text splitting*** ed **embeddings** rappresentano sempre un aspetto critico nella realizzazione dei sistemi RAG.

In particolare, è importante garantire che i singoli **chunk** mantengano  un significato semantico completo, cioè non abbiano un inizio o una fine che interrompano la sintassi di una proposizione, rendendo incomprensibile la frase.

Un altro aspetto da considerare è che i testi dei **chunk** siano semanticamente correlati al contesto di origine.

Ad esempio estraendo a caso una pagina dal manuale di un prodotto, dove viene mostrato il modo in cui effettuare il log-in ad una applicazione, potrebbe non essere più evidente a quale prodotto specifico si riferisca l'istruzione.

Ovviamente se nella pagina è presente il titolo del prodotto o sono presenti header e footer in cui è citato il prodotto, il capitolo o il manuale di origine, questi frammenti di informazione possono orientare il LLM a comprendere meglio il contesto.

Per questo motivo sono stati sviluppati diversi algoritmi di **text splitting** che agiscono creando **chunk** di buona qualità, sfruttando anche le meta informazioni presenti nei formati di file come il PDF o il formato Office.

LangChain offre specifiche funzioni **loader** ottimizzate sui formati commerciali più diffusi.

Tuttavia, per evitare di complicare il processo di popolamento del **vectorstore** una buona pratica, che si è affermata nel settore, è quella di preprocessare tutti i file, convertendoli in formato PDF.

Il PDF è un formato sempre disponibile su tutti gli strumenti Office ed è un formato che preserva o, in molti casi, genera le meta informazioni, utilissime agli algoritmi di **text splitting**

Per questo motivo, nel nostro prototipo abbiamo utilizzato esclusivamente il formato PDF.

### Inizializzazione del vectorstore

Una volta che abbiamo definito la classe **`VectorStoreManager`**, possiamo creare fisicamente il nostro **vectorstore**.

La cella che segue, imposta il path della directory in cui memorizzare fisicamente il DB vettoriale e inizializza il **vectorstore** creando l'oggetto **manager** che rappresenterà il nostro **vectorstore**.

I casi d'uso in cui possiamo eseguire la cella di codice sono due:

1. Vogliamo creare un nuovo *vectorstore* vuoto.
2. Abbiamo già un DB vettoriale dentro una cartella sul file system e vogliamo semplicemente inizializzare l'oggetto **manager** affinché *punti* a tale DB.

Nel primo caso, sarà sufficiente impostare il path desiderato come valore della variabile ***persist_dir*** e lanciare l'esecuzione del codice.
Verrà creata una cartella nel path indicato e all'interno saranno presenti i file della struttura DB, ma saranno privi di dati.

Nel secondo caso, sarà sufficiente impostare il valore della variabile **`persist_dir`** con il path della cartella in cui è presente il DB esistente e lanciare l'esecuzione del codice.

Più avanti mostreremo come usare il comando **zip** per salvare sul disco del proprio PC la cartella contenente il DB generata sull'area file system di questo notebook, e come ripristinare dal file sul proprio PC una cartella con il DB nell'area file system di questo notebook.

In [ ]:

# Specifica la directory di persistenza
persist_dir = "chroma_persist"

# Inizializza il manager
manager = VectorStoreManager(persist_dir)



### Gestione del vestorstore

Vediamo ora alcuni esempi di codice che permettono di inserire, eliminare o aggiornare documenti nel **vectorstore** che abbiamo creato.

Il caso più semplice è quello mostrato dalla cella di codice che segue.

#### Aggiunta di un documento singolo

Prendiamo un singolo documento PDF, carichiamolo sul file system di questo notebook, prendiamo nota del path e lanciamo l'esecuzione della cella, dopo aver aggiornato correttamente il path del documento.

Il file sarà frammentato in **chunk**, sottoposto ad **embeddings** e aggiunto al **vectorstore**.

In [ ]:
# Aggiunge un nuovo documento
nuovo_documento_path = "/content/Z9RG_(It)05.pdf"  # Sostituisci con il percorso reale
num_chunks_aggiunti, id_nuovo_documento = manager.add_document(nuovo_documento_path)
if id_nuovo_documento:
    print(f"Numero di chunk aggiunti: {num_chunks_aggiunti}")
    print(f"ID del nuovo documento: {id_nuovo_documento}")



#### Eliminazione di un documento dal vectorstore

Se vogliamo eliminare dal **vectorstore** un documento precedentemente caricato, dobbiamo conoscere il suo ID univoco.

Nel nostro banale algoritmo di generazione degli ID, l'ID è semplicemente la stringa con il nome del file, senza l'intero path.

In ogni caso, la lettura del file **id_mapping.json** presente nella cartella principale del DB mostra gli ID dei documenti presenti nel DB.

Sarà sufficiente modificare il codice seguente con l'ID del documento che vogliamo rimuovere e lanciare l'esecuzione della cella di codice.

In [ ]:
manager.remove_document("Z9UMEUR_(It)03.pdf")

#### Inserimento batch di più docuemnti

Per un popolamento iniziale del **vectorstore** abbiamo predisposto un metodo che carica tutti i documenti presenti in una cartella sul file system.

Creiamo una cartella sul file system di questo notebook con il nome che riteniamo più opportuno, copiamo in questa cartella i documenti PDF che vogliamo aggiungere al **vectorstore** modifichiamo il codice seguente con il path della cartella e lanciamo l'esecuzione della cella di codice.

I file presenti nella cartella saranno caricati uno ad uno nel DB con la tecnica già descritta.

In [ ]:
documents_directory = "/content/KnowledgeBase"  # Sostituisci con il percorso reale
manager.load_documents_from_directory(documents_directory)

#### Aggiornamento di un documento

Per aggiornare un documento presente nel DB sarà sufficiente eliminarlo e caricare la nuova versione.

Nella definizione della classe **`VectorStoreManager`** abbiamo anche incluso un metodo che fa esattamente questo.

### Backup e ripristino del vectorstore

Se vogliamo salvare sul nostro PC il **vectorstore** possiamo eseguire l'istruzione Linux seguente e poi scaricare dall'area file system di questo notebook il file .zip contenente tutta la cartella di persistenza del DB.

In [ ]:
!zip -r chroma_persist.zip /content/chroma_persist/


Per ripristinare nell'area file system di questo notebook un DB a partire dal file .zip salvato sul nostro PC, possiamo caricare tale file nel file system del notebook ed eseguire il seguente comando Linux.

In [ ]:
!unzip chroma_persist.zip -d .

## Recupero del retriever

In LangChain un **retriever** è un oggetto specializzato nel recupero di informazioni mediante **query semantiche**.

Il **retriever** è un oggetto **runnable** (spiegheremo fra poco cosa vuol dire) che può essere facilmente assemblato in una **chain** o trasformato in un **tool**.

Tutti gli oggetti di tipo **vectorstore** in LangChain offrono un metodo chiamato as_retriever, che permette di generare un oggetto **retriever** associato al **vectorstore**.

Nel nostro caso, poiché abbiamo definito una classe wrapper per il **vectorstore** abbiamo anche incluso un metodo per recuperare il relativo **retriever**.

Il codice che segue, recupera il **retriever** dall'oggetto **manager** creato in precedenza e lo assegna all'oggetto **`retriever`**.

In [ ]:
# Ottenere il retriever
retriever = manager.get_retriever()

Una volta ottenuto il **retriever** possiamo sperimentare il suo funzionamento chiamando il metodo **`invoke`**, offerto da tutti i **runnable** LangChain, passando una **query** qualsiasi.

Eseguendo la cella che segue, eventualmente modificango la stringa di query, possiamo vedere il testo grezzo estratto dalla ricerca semantica sul **vectorstore**.

Quando invochiamo il **retriever** quello che accade dietro le quinte è il processo seguente:

1. Viene invocato il modello di embeddings impostato all'atto della creazione del vectorsore per calcolare il **vettore di embeddings** della **query**. Nel nostro caso viene invocato il servizio AWS Bedrock passando la query al modello **Amazon Titan Embeddings text V2**;
2. Il **vettore di embeddings** della **query** viene usato per effettuare la ricerca, e il recupero, dei **chunk** più affini, secondo un criterio algoritmico chiamato **similarity** che usa una formula matematica detta del **minimo seno**.
3. I **chunk** recuperati vengono confezionati in un tipo di dato adatto ad essere gestito da una **chain** LangChain.

Il tutto avviene senza che lo sviluppatore debba preoccuparsi dei dettagli.

In [ ]:
retriever.invoke("esiste l'esposizione spot?") # Modificare la query con quella desiderata

# Sezione 2 - Costruzione di un ChatBot RAG stateless mediante chain

In questa sezione viene mostrato come realizzare lo schema base di un processo **RAG** utilizzando il framework **LangChain**.

Il **ChatBot** che viene implementato in questa sezione è di tipo **stateless**, cioè non è influenzato da eventuali precedenti chiamate da parte dell'utente.

In uno scenario di produzione, ogni invocazione del servizio attiverebbe un nuovo *thread* indipendente che nasce con la chiamata e termina con la restituzione della risposta.

Per motivi puramente didattici, abbiamo realizzato questo ChatBot assemblando i diversi componenti in una ***chain*** definita ad hoc.

Come vedremo successivamente, per schemi di processo consolidati, **LangChain** offre funzioni **built-in** che semplificano notevolmente lo sviluppo del software.

Nella cella seguente sono richiamate le classi e i metodi che useremo per costruire il ChatBot.

In particolare:

- **PromptTemplete** è una classe di LangChain, molto potente, che permette di costruire facilmente e dinamicamente i prompt.
- **StrOutputParser** è una classe LangChain di utilità che permette di estrarre dagli oggetti restituiti come risposta dai LLM il solo campo contenente la stringa di testo.
- **RunnablePassthrough** è un metodo di utilità di LangChain che consente di trasferire in output, senza alcuna modifica, un oggetto ricevuto in input, in un qualunque elemento di una chain.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate

## Scelta del modello Chat LLM

L'istruzione che segue, crea un oggetto, chiamato **llm** che rappresenta il client di accesso al modello LLM che abbiamo deciso di utilizzare.

Prima di creare il modello viene importata la classe **ChatBedrock** che gestisce l'integrazione di **LangChain** verso i cosiddetti **modelli fondazione di base** offerti da **Amazon** tramite il servizio **AWS Bedrock**.

In [ ]:
from langchain_anthropic import ChatAnthropic
llm = ChatAnthropic(model="claude-3-5-sonnet-20241022")


In [ ]:
llm.invoke("esiste l'esposizione spot?")

## Costruzione della chain e dei componenti elementari

Prima di costruire la **chain** vera e propria, abbiamo bisogno di alcuni componenti.

La cella che segue dichiara una funzione di utilità, che servirà a compattare i **chunk** di testo, recuperati dal **retriever**, in una unica stringa di testo.

Infatti, provando ad invocare il **retriever**, come mostrato nella precedente sezione, l'oggetto di ritorno è una lista di oggetti strutturati (si tratta di oggetti della classe LangChain *document*), che nel campo **.content** contengono la stringa di testo che costituisce il contenuto che ci interessa.

La funzione **format_docs** riceve in input una *lista* di oggetti di tipo *LangChain document* e, usando il metodo standard delle stringhe Python *.join* li aggrega in una unica stringa, in cui tra un blocco di testo e l'altro sono inseriti due caratteri *newline*.

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)



La cella di codice che segue mostra l'effetto della funzione appena definita

In [ ]:
result = retriever.invoke("esiste l'esposizione spot?")
print('result =',result)
print("context: ",format_docs(result))

Il codice seguente crea l'oggetto *runnable*, chiamato **prompt**, di tipo **PromptTemplate**.

I *prompt template* sono un ulteriore esempio dei potenti strumenti offerti da LangChain.

La costruzione accurata dei prompt da inviare ai LLM è un elemento fondamentale per garantire l'efficacia delle applicazioni.

Il **prompt engineering** è oggi una vera e propria disciplina.

La stringa **template** contiene la struttura base del prompt, le istruzioni fisse in italiano, che dovranno sempre essere inviate al modello e due *placeholder* chiamati **question** e **context**.

Il costruttore **PromptTemplete** genera l'oggetto **prompt** specificando che  sono previste due variabili di input, chiamate **context** e **question**.

Il **prompt effettivo** (prompt value) che sarà inviato al modello LLM in questo caso è molto semplice. Le istruzioni di comportamento, la domanda dell'utente e il contesto sono raggruppati in un unico messaggio che sarà inviato con il **ruolo** di **user**.

Nel paragrafo successivo vedremo come strutturare un prompt più efficace utilizzando messaggi diversificati per ruolo.

In [ ]:
template = """Sei un assistente che aiuta l'utente a trovare soluzioni alle procedure di uso del prodotto.
Usa come contesto i contenuti recuperati dal manuale utente. Per la risposta usa anche le tue competenze generali,
ma se il contesto non contiene elementi pertinenti consiglia di rivolgersi ad un assistente umano.

Domanda: {question}
Contesto: {context}"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)


L'istruzione seguente genera l'oggetto **rag_chain** che è il nostro **ChatBot RAG**

È giunto il momento di fornire alcuni dettagli sui concetti di **runnable** e **chain**, che costituiscono il cuore e l'essenza del framework **LangChain**.

In LangChain un **runnable** è un oggetto assimilabile ad una **funzione eseguibile**, ma in realtà molto più potente.

La classe **runnable** prevede una serie di metodi per avviare l'esecuzione della funzione in scenari e contesti diversi.

Il metodo **.invoke** ad esempio è quello utilizzato nelle chiamate automatiche che avvengono durante l'esecuzione di una **chain**, ma in realtà è possibile lanciare l'esecuzione sincrona di un *runnable* anche attraverso i metodi **stream** e **batch**. Inoltre, esiste anche la possibilità di lanciare i *runnable* con modalità asincrone, usando i metodi **ainvoke**, **abatch** e **astream**.

Ciascun metodo prevede specifici schemi per l'input e l'output.

La nostra **rag_chain** è costruita attraverso il formalismo LCEL di LangChain, che sfrutta l'operatore '|' (pipe).

La *chain* è a sua volta un oggetto **runnable** per cui sarà lanciato con il metodo **invoke**.

Come parametro, passeremo una stringa, che rappresenta la query dell'utente.

Questa stringa viene fornita in input al primo elemento della chain che è la definizione esplicita di un dizionario Python:

 `{"context": retriever | format_docs, "question": RunnablePassthrough()}`

 Il dizionario contiene due chiavi, **context** e **question**, che sono proprio le due variabili attese in input dal prompt template **prompt** che abbiamo definito prima.

 Alla chiave **context** viene associato l'output di una mini chain costituita da due elementi: **`retriever | format_docs`**.

 Il primo elemento è l'oggetto **retriever** che abbiamo costruito nella sezione deicata al **vectorstore** e il secondo elemento è la funzione di utilità che abbiamo costruito sopra.

 Il valore della stringa che contiene la **query** dell'utente, viene quindi ricevuta in input dal **retriever** che effettua la sua ricerca semantica restituendo la lista di oggetti *LangChain document* corrispondenti ai chunk di testo recuperati. Questa lista viene passata in input alla funzione **format_docs** che restituisce una singola stringa che viene associata alla chiave **context**.

 Alla chiave **question** viene associato l'output della funzione **RunnablePassThrough** che è esattamente il valore fornito in input, cioè il testo della **query** dell'utente.

 Questo dizionario è l'input al secondo elemento della **rag_chain** che è il prompt template **prompt**, che riceve le sue due variabili e fornisce in output un **prompt value** che contiene: istruzioni, contesto e query dell'utente.

 Questo prompt viene passato in input al successivo elemento della chain, che è l'oggetto **llm**.

 A questo punto sarà invocato il LLM e la risposta ricevuta sarà inviata all'ultimo elemento della catena, che è la funzione **StrOutputParser** che restituirà semplicemente il messaggio di risposta.


In [ ]:
SistemaRAG = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

La **chain** che abbiamo appena definito è un **runnable** che possiamo eseguire come mostrato nel codice seguente.

La stringa in output è in genere costituita da testo formattato mediante il linguaggio **MarkDown**, per cui abbiamo utilizzato delle funzioni di libreria, specifiche per i notebbok Jupyter, in modo da visualizzare il testo in modo formattato.

In [ ]:
query = "posso impostare l'esposizione spot?"
answer = SistemaRAG.invoke(query)

# stampo la risposta con il corretto formato
from IPython.display import display, Markdown
from google.colab import output

display(Markdown(answer))

## Streaming

Come risulta evidente dall'esecuzione del nostro ChatBot, la risposta da parte di un LLM non è immediata.

Per gestire il problema *psicologico* di una attesa che può apparire eccessiva all'utente esistono diverse tecniche.

Una possibile tecnica è quella di mostrare in output le singole parole man mano che vengono generate dal LLM.

La maggior parte dei LLM, infatti, fornisce la risposta in una modalità chiamata **streaming**.

Nella cella seguente viene mostrato come eseguire la **chain** in questa modalità.

In [ ]:
for chunk in SistemaRAG.stream("Come imposto l'esposizione matrix?"):
    print(chunk, end="", flush=True)

## Semplificazione del codice con le funzioni built-in di LangChain

Nel codice precedente, per motivi didattici, abbiamo mostrato come realizzare una **chain** usando il formalismo ***LCEL***.

Questo metodo è utile quando si devono costruire workflow molto complessi per applicazioni particolari.

Quando si implementano sistemi per casi d'uso comuni, sui quali esistono delle **best practices** consolidate, molto probabilmente il framework LangChain offre delle funzioni predefinite **built-in** che costruiscono la **chain** nel rispetto di queste **best practices**.

Essendo il ChatBot RAG un tipico esempio di problema consolidato, possiamo implementare lo stesso ChatBot stateless in modo molto più immediato nel modo seguente.

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "Sei un assistente che aiuta l'utente a trovare soluzioni alle procedure di uso del prodotto. "
    "Usa come contesto i contenuti recuperati dal manuale utente. "
    "Per la risposta usa anche le tue competenze generali,"
    "ma se il contesto non contiene elementi pertinenti consiglia di rivolgersi ad un assistente umano."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


Il metodo **create_stuff_documents_chain** costruisce automaticamente una chain in cui un insieme di elementi di testo (*documents* nella terminologia LangChain) viene passato così come'è (*stuff* ha il significato di inserimento) ad un LLM. I parametri di input sono il LLM e un prompt template che deve avere almeno una variabile denominata "context".
La chain prodotta da questo metodo si aspetta in input un dizionario che abbia obbligatoriamente una chiave chiamata "context", oltre ad eventuali chiavi definite nel prompt template.
L'output della chain generata dipende dal parametro OutParser che eventualmente viene passato. Non avendo passato questo parametro, il default sarà il parser *StrOutputParser* per cui l'output sarà una semplice stringa di testo.

Questo metodo, come si vede, costruisce gran parte della chain che abbiamo definito in modo esplicito precedentemente.

Il metodo **create_retrieval_chain** aggiungfe ad una chain un passo di **retrieval**. L'output restituito è un dizionario in cui la risposta è associata ad una chiave chiamata "**answer**".

Per cui passando il nostro oggetto **retriever** e la chain generata dal metodo precedente abbiamo costruito la nostra **rag_chain**.

Anche il metodo utilizzato per costruire il **prompt template** è leggermente diverso.

In questo caso il template è strutturato come sequenza di messaggi e non come unico messaggio. Il **prompt value** sarà quindi composto da diversi messaggi che saranno inviati al **LLM** con **ruoli** differenti.

Questa nuova chain generata dalle funzioni *built-in*, si aspetta quindi in input un **dizionario** che contiene la chiave "**`input`**" valorizzata con la query e restituisce in output un **dizionario** in cui la stringa di risposta si trova come valore associato ad una chiave chiamata "**`answer`**".

Per invocare la chain e visualizzare il risultato mediante streaming possiamo farlo nel modo seguente.


In [ ]:
for chunk in rag_chain.stream({"input": "Come imposto l'esposizione matrix?"}):
    if "answer" in chunk: # Poiché uso lo streaming la chiave answer compare dopo alcuni chunk per cui è necessario il test
      print(chunk["answer"], end="", flush=True)



Nel seguito una alternativa di esecuzione senza streaming.


In [ ]:
response = rag_chain.invoke({"input": "come imposto l'esposizione matrix?"})


# stampo la risposta con il corretto formato
from IPython.display import display, Markdown
from google.colab import output

display(Markdown(response["answer"]))

## Sezione 3 - Costruzione di un ChatBot Rag conversazionale

Il ChatBot realizzato nelle sezione precedente risponde esclusivamente alla **query** fornita al momento dell'invocazione senza tenere in considerazione eventuali altri messaggi e risposte scambiati in precedenza.

Un **ChatBot conversazionale**, al contrario, è un assistente in grado di rispondere tenedo in considerazione anche eventuali messaggi scambiati in precedenza.

Un assistente di questo tipo deve quindi necessariamente operare mantenendo attive delle **sessioni**. La prima volta che l'utente richiede assistenza formulando una domanda, deve essere inizializzata una nuova **sessione** in modo che tutte le successive richieste avvengano nell'ambito di questa specifica sessione.

In uno **scenario di produzione** deve essere definito il meccanismo per la gestione delle **sessioni**: quando iniziare una nuova sessione, a cosa legare la sessione (ad esempio associare la sessione della chat di assistenza alla sessione di accesso dell'utente al prodotto), quando chiudere una sessione (per iniziativa dell'utente o perché l'assistente ritiene che l'ultima risposta fornita sia definitiva o per timeout, ...

Nel caso del nostro prototipo ci limiteremo ad impostare un semplice meccanismo per identificare la sessione corrente, a puro scopo esemplificativo.

Tutti i messaggi, **query** e **risposte** scambiati durante una sessione costituiscono una **chat history**.

Un **ChatBot Conversazionale** dovrà quindi mantenere uno stato che terrà conto di **session ID** e **chat history**.

Nella definizione dei **prompt** occorrerà tenere conto del fatto che una **query** posta dall'utente possa implicitamente fare riferimento a messaggi o risposte presenti nella **chat history**.

Lo schema **RAG** ne risulta particolarmente affetto, in quanto un'eventuale **query**, la cui interpretazione dipenda dal contesto, non è immediatamente utilizzabile per la **ricerca semantica**.

Un esempio potrebbe essere la seguente query: "*Come ne inserisco una?*".

**A cosa si riferisce l'utente?**

In un contesto in cui la **chat history** fosse la seguente:

- ***Utente***: *Il sistema gestisce la possibilità di creare più di una utenza?*

- ***Assistente***: *Si, certamente. L'amministratore può definire utenze diverse con differenti profili di sicurezza.*

la **query** andrebbe interpretata: "*come posso inserire una nuova utenza?*"

Al contrario, nel caso in cui la **chat history** fosse la seguente:

- ***Utente***: *Si possono gestire le gare?*

- ***Assistente***: *Si, certamente. Il sistema permette la difinizione di gare con diverse modalità di svolgimento.....*

la **query** andrebbe interpretata: "*come posso inserire una nuova gara?*"

Pertanto, per gestire l'operazione di **retreival** dobbiamo per prima cosa costruire un tipo di **retreiver** che tenga conto della **chat history**.

Il flusso delle operazioni diviene quindi leggermente più complicato.

Possiamo sfruttare il *LLM* per riformulare la *query* ambigua in modo non ambiguo e quindi usare questa nuova query per effettuare l'operazione di *retreival*.

**LangChain** offre una potente classe, specificamente progettata per gestire problemi di questo tipo.

La cella che segue mostra come realizzare un **retreiver** che tiene conto della **chat history** utilizzando il metodo LangChain **create_history_aware_retriever** e la tecnica appena descritta.

Per prima cosa viene creato uno specifico **prompt template** per chiedere al **LLM** di creare una nuova **query** che sia significativa senza dover ricorrere alla **chat history**.

Quindi, viene creato l'oggetto **`history_aware_retriever`** passando al metodo **create_history_aware_retriever** l'oggetto **llm**, che rappresenta il client di accesso al modello LLM che abbiamo selezionato, l'oggetto **retreiver**, che è il *retreiver* classico recuperato dal nostro **vectorstore** e il **prompt template** che contiene le istruzioni per il riformulamento della **query**.

Si noti che il **prompt template** che abbiamo definito contiene un **placeholder** per l'inserimento dell'intera **chat history**.

Per consentire la costruzione del template attraverso l'impiego del **placeholder** è stato usato il metodo LangChain **ChatPromptTemplate.from_messages**.




In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# Testo del prompt di sistema che istruisce il LLM su come riformulare la query
contextualize_q_system_prompt = (
    "Data la chat history e l'ultima query dell'utente,"
    "che potrebbe anche fare riferimento al contesto nella chat history, "
    "genera una nuova query che possa essere compresa "
    "senza la chat history. NON rispondere alla domanda, "
    "ma riformula la domanda, se necessario, oppure restituiscila così come è."
)

# Definizione del prompt template per ottenere la query contestualizzata
# MessagePlaceholder prevede che venga inserito in quel punto del template una sequenza di messaggi
# associati alla chiave 'chat_history'
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


Ovviamente, anche il **prompt template** che useremo per invocare il LLM chiedendo di rispondere alla **query** dell'utente dovrà essere modificato per inserire la **chat history**.

Nella cella che segue, viene costruito il nuovo **prompt template**, usando il metodo **ChatPromptTemplate.from_messages** a cui passiamo come argomenti:

- Un template di messaggio di sistema che prevede l'inserimento di una variabile **context**
- Un **placeholder** per l'inserimento della **chat_history**
- Un template di messaggio utente che prevede l'inserimento di una variabile **input**



In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "Sei un assistente che aiuta l'utente a trovare soluzioni alle procedure di uso del prodotto. "
    "Usa come contesto i contenuti recuperati dal manuale utente. "
    "Per la risposta usa anche le tue competenze generali,"
    "Se ritieni che nel contesto fornito le informazioni siano molto insufficienti, consiglia di rivolgersi ad un assistente umano."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)




A questo punto, con due istruzioni, possiamo creare la **chain** utilizzando le consuete funzioni **built-in** offerte da LangChain.

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

## Gestione della chat history e della sessione

Fino a questo punto abbiamo sempre fatto riferimento alla **chat history** attraverso il nome del suo segnaposto: **chat_history**.

Come implementiamo fisicamente questa entità?

Come interviene il concetto di sessione?

In un contesto multi utente e multi threading, le chat history devono essere riferite a singoli thread o sessioni.

Un modo semplice e diretto di prototipare questo aspetto è quello di utilizzare un dizionario Python, definendo delle chiavi corrispondenti ai singoli identificatori di sessione e associare a tali chiavi la sequenza di messaggi che si riferisce a quella particolare sessione.

LangChain offre una serie di classi molto utili per gestire le chat history.

Nel codice seguente sfruttiamo queste classi, mostrandone l'utilizzo.

Per prima cosa viene inizializzato il dizionario chiamato **store** come dizionario vuoto.

Viene quindi definita una funzione, chiamata **get_session_history** che riceve una stringa come parametro di input, corrispondente ad un identificatore di sessione, e restituisce un oggetto della classe LangChain **BaseMessageHistory**.

**BaseMessageHistory** è una classe astratta di LangChain che specifica l'interfaccia generale degli oggetti che costituiscono strutture idonee a memorizzare cronologie di messaggi. L'interfaccia prevede metodi standard per aggiungere e rimuovere messaggi dalla cronologia.

La funzione accede alla variabile globale **store* e verifica se la stringa in input è presente come chiave nel dizionario.

Se non è presente, viene creata con tale stringa una chiave nuova nel dizionario, a cui viene associato un nuovo oggetto della classe **ChatMessageHistory**.

La classe **ChatMessageHistory** è una semplice implementazione della classe astratta **BaseMessageHistory** che mantiene in memoria la cronologia dei messaggi.

*In uno scenario di produzione avremmo usato una implementazione di tipo persistente, ad esempio avremmo potuto usare la **SqlChatMessageHistory** che memorizza la cronologia di messaggi in un DB SQLLite.*

Infine, il valore associato alla chiave passata come parametro di input viene restituito.

In pratica la funzione restituisce l'oggetto che offre i metodi previsti dall'interfaccia astratta standard di LangChain per gestire la memorizzazione della chat history associata all'ID di sessione.

A questo punto interviene l'utilissima classe **RunnableWhithMessageHistory** che consente di trasformare una **chain** relativa a un ChatBot in una nuova **chain** che gestisce in modo corretto la **chat history**.

I parametri da fornire al costruttore **RunnableWithMessageHistory** sono:

- la **chain** di partenza
- la funzione da usare per recuperare l'oggetto **BaseChatMessageHistory** partendo dall'ID di sessione (la funzione *get_session_history* è stata definita proprio per questo scopo)
- La chiave che identifica il messaggio di input, cioè la *query* dell'utente
- La chiave che identifica il *placeholder* della chat history
- La chiave che identifica il messaggio di output, cioè la risposta all'utente

In questo modo viene creata la nuova **chain** **conversational_rag_chain** che gestisce sessioni e chat history.

Questa nuova chaincomprende tutte le operazioni necessarie a gestire la cronologia dei messaggi. In particolare si occupa di recuperare la cronologia dei messaggi per poterli fornire al primo elemento della chain di base e al termine della esecuzione della chain di base aggiunge la risposta fornita alla cronologia dei messaggi.

La **conversational_rag_chain**, come tutti i **runnable** generati tramite il metodo **RunnableWhithMessageHistory** prevedono che venga passato un parametro di configurazione corrispondente alla chiave predefinita "session_id" con cui recuperare la **BaseChatMessageHistory** associata alla specifica sessione.



In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)


Per provare il funzionamento del nuovo **ChatBot RAG conversazionale** possiamo usare il codice seguente, che crea un loop consentendo all'utente di inserire più domande, costruendo una conversazione.

La chain viene invocata passando due parametri:

- un dizionario contenente la chiave "input" valorizzata con la query inserita dall'utente
- il parametro **config** valorizzato con un dizionario in cui la chiave "**configurable** è associata ad un ulteriore dizionario che abbina la chiave "**session_id**" ad un identificativo fittizio che abbiamo arbitrariamente definito. LangChain offre la possibilità di definire parametri di configurazione personalizzati chiamati **configurable**, nel nostro caso abbiamo scelto di usare questo metodo per passare alla chain l'identificativo della sessione corrente. I *configurable* sono un'altra caratteristica, molto potente, che consente di creare chain adatte ad un contesto di produzione.

In [ ]:

while True:
  domanda = input("> ")
  if domanda == "stop":
    break
  risposta = conversational_rag_chain.invoke({"input": domanda}, config={"configurable": {"session_id": "abc123"}} )
  print("\n",risposta["answer"],"\n")

Avremmo potuto, ovviamente, creare la **chain** conversazionale  in modo esplicito, anziché utilizzare gli strumenti built-in di LangChain, ma il codice sarebbe stato molto più complesso e il processo di sviluppo più lungo.

L'uso del framework permette al progettista di concentrarsi sugli aspetti che contano e che sono distintivi delle tecniche di utilizzo dei LLM.

# Sezione 4 - Realizzazione di un agente

La soluzione fornita nella sezione precedente è già ottimale e potrebbe essere la base per confezionare il servizio da distribuire in produzione.

Tuttavia, presenta alcuni potenziali svantaggi, derivanti dal fatto che il **workflow** della **chain** è fisso.

Un utente potrebbe esordire nella sua conversazione con un messaggio generico del tipo: "*Ciao, sono Stefano*."

Se proviamo ad inviare un tale messaggio alla chain precedente, osserveremo che il ChatBot fornirà una corretta risposta di circostanza del tipo: "*Ciao Stefano, come posso aiutarti?*", tuttavia, conoscendo il flusso che abbiamo implementato, la nostra chain avrà comunque effettuato una operazione di **retrieval**, che implica l'invocazione del servizio di **embeddings** che ha un costo, che invece, avrebbe potuto essere evitato.
Quando poi viene invocato il LLM per la risposta finale, sarà inviato in input anche l'inutile contesto recuperato dal *retreiver*, con aggravio di token e quindi di costo.

Inoltre, se otteniamo una risposta di circostanza corretta, ciò è dovuto alla buona qualità raggiunta dai recenti LLM che non si lasciano confondere dal contesto non pertinente alla query. Un modello meno performante avrebbe potuto generare una risposta più bizzarra.

Come possiamo risolvere questo tipo di problemi?

Un metodo è quello di creare un flusso di elaborazione più articolato, che preveda delle possibili diramazioni condizionali.

Potremmo verificare il tipo di query inserita e decidere di non effettuare il *retreival* se questo non risulti necessario.

Ma come può il nostro software analizzare la query per prendere questa decisione?

Potremmo utilizzare proprio le capacità di ragionamento del LLM per effettuare questa valutazione.

Quello a cui stiamo pensando è un software il cui flusso di elaborazioni ha delle diramazioni determinate dalla risposta di un LLM, e quindi in ultima analisi, dalla sua capacità di ragionamento.

Un sistema di questo tipo è definito **agente**.

L'implementazione di **agenti** pone una serie di problemi e, sebbene sia possibile anche usando sistemi LLM di generazione precedente, è una tecnica possibile soprattutto con i Chat LLM di ultima generazione che prevedono, e sono quindi stati addestrati per prevedere, l'impiego di **tool**.

Un modello di questo tipo, come quello utilizzato in questo prototipo, è in grado di valutare se una *query* possa essere immediatamente gestita, oppure se sia più utile invocare uno o più **tool**, tra quelli messi a disposizione, per recuperare ulteriori informazioni.

Tramite il potente formalismo di LangChain è possibile sviluppare un flusso di elaborazione di diverse **chain** articolate tra loro, tuttavia, questa strada richiede una buona conoscenza del problema e una certa dose di lavoro.

In realtà il settore della ricerca in questo campo ha già prodotto delle *best practices* ottimizzate per i diversi LLM presenti sul mercato e LangChain offre un supporto potente e specifico per la creazione degli agenti.

In particolare, il framework aggiuntivo **LangGraph** è stato rilasciato proprio per agevolare lo sviluppo di flussi di elaborazione di tipo **agentico**.

Nel seguito costruiremo una **soluzione agentica** al nostro problema utilizzando questi strumenti.

## Trasformare il *retreiver* in un tool

Per realizzare un processo **RAG** tramite un agente dobbiamo trasformare il nostro **retreiver** in un **tool**.

Ogni moderno Chat LLM prevede la possibilità di fornire nel *prompt* una lista di **tool**.

L'interfaccia è diversa per ogni LLM, sia nelle modalità di passaggio della lista dei tool sia nella struttura stessa di questo parametro e degli stessi tool.

I **tool** sono funzioni o API, locali o remote, che devono avere una interfaccia standard e una descrizione in linguaggio naturale, ovviamente ogni LLM ha le sue peculiari specifiche, in modo che il LLM possa decidere se richiederne l'invocazione prima di fornire una risposta definitiva alla query dell'utente.

LangChain fornisce un modello astratto ed elegante per tutto questo, che ci permette di definire **tool** ed **agenti** in modo indipendente dalle specifiche indotte dal particolare LLM che andremo ad utilizzare.

Il **retreiver**, quindi, non sarà sempre invocato, in quanto passaggio fisso all'interno di una **chain**, ma sarà un **tool** a disposizione del LLM che avrà facoltà di decidere se invocarlo o meno.

LangChain fornisce un metodo di libreria per generare un **tool** da un **retreiver**.

La cella di codice seguente crea il nostro **retreiver tool**.

Come si vede, nella definizione del **tool** è necessario specificare un identificatore e una descrizione.

Entrambi i campi devono essere definiti con attenzione e devono essere ampiamente descrittivi del compito e delle funzionalità assegnate al **tool**.

Questi campi diventano parte del **prompt** e quindi sono usati nel **ragionamento** del LLM.

Il LLM decide di utilizzare il **tool** perché dalla sua descrizione e dal suo *nome* risulta evidente che possa essere utile per elaborare la risposta!

In [ ]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "product_document_retriever",
    "Ricerca e restituisce testi significativi dalla documentazione sul prodotto.",
)
tools = [tool]


## Creazione dell'agente

Il nostro agente deve svolgere più o meno il seguente flusso di lavoro:

- Riceve un input sotto forma di messaggi (la query iniziale dell'utente, oppure la query con una cronologia di messaggi, ed eventualmente un contesto)
- Chiama il modello LLM per decidere se utilizzare il **retriever**
- Se il modello decide di usare il **retriever**, lo invoca e passa i risultati nuovamente al modello aggiungendo eventualmente riformulando il prompt (questi ultimi due passi potrebbero essere ripetuti più volte)
- Se il modello non richiede l'uso del tool, termina e risponde all'utente.

Questo schema per il flusso di elaborazione è di tipo abbastanza comune ed è stato generalizzato in un modello di agente chiamato **ReAct**, dalla crasi di **Reasonining** e **Act**.

Un **agente**, ovviamente, deve gestire uno stato interno che comprenderà, oltre ad un ID di sessione, la cronologia dei messaggi della chat ma anche le sequenze dei messaggi di risposta e i nuovi prompt generati nei cicli intermedi.

**LangGraph** prevede una funzione **prebuild** per generare proprio un **agente** di tipo **ReAct**.

Quello che serve fornire a questo metodo **prebuild** è semplicemente:

- Un oggetto runnable llm
- Una lista di tool
- Un oggetto della classe **Checkpointer** per gestire la persistenza dello stato.

Nella cella che segue viene creato l'oggetto **agent_executor** utilizzando **create_react_agent** e passando come *checkpointer* l'oggetto **memory** creato con il costruttore della classe **MemorySaver** che costituisce una semplice implementazione in memoria di un *checkpoint saver* LangGraph.

Utilizziamo un **prompt di sistema** per modificare lo stato interno dell'agente **prebuild** in modo da personalizzare l'agente al nostro obiettivo.

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

system_prompt = ("Sei un assistente di nome Marta. Il tuo scopo è fornire assistenza nell'uso del prodotto."
                  "Il tuo compito è ridurre gli accessi al contact center di assistenza del prodotto."
                  "Usa sempre le informazioni recuperate dai manuali."
                  "Se ti vengono poste domande che non riguardano il prodotto, la prima volta rispondi"
                  "in modo sintetico usando le tue capacità e comunque ricorda all'utente chi sei e che tipo di domande puoi ricevere."
                  "Per domande riguardanti la fotografia in generale e le fotocamere nikon, puoi rispondere"
                  "usando anche la tua conoscenza generale."
                  "Usa sempre un tono cordiale e sii cortese."
                )
memory = MemorySaver()
agent_executor = create_react_agent(llm, tools, checkpointer=memory, state_modifier =system_prompt)


Nella cella che segue è contenuto il codice per provare il funzionamento del nostro **agente**.

L'input atteso dall'agente creato mediante il **prebuild** di **LangGraph** è costituito da un dizionario contenente almeno la chiave "**messages**" che viene valorizzata con i messaggi di **input**.

Nel nostro caso, la struttura dell'input è costituita dalla sola **query** dell'utente, per cui utilizziamo la classe helper **HumanMessage** di LangChain per creare la struttura del messaggio utente.

In aggiunta, viene passato al metodo invoke dell'agente un **configurable** che definisce l'identificativo di sessione. Abbiamo usato anche questa volta una stringa arbitraria.

In questo caso, il nome predefinito per la chiave che specifica l'ID di sessione è "**thread_id**".

Si noti che la struttura della risposta fornita dall'agente è un dizionario che contiene la chiave **messages**.

Il valore associato a questa chiave è una lista che contiene tutti i messaggi della chat history compresi quelli generati dal LLM nei cicli intermedi.

La risposta all'utente è l'ultimo messaggio di questa lista.

In [ ]:
from langchain_core.messages import HumanMessage

from IPython.display import display, Markdown
from google.colab import output


config = {"configurable": {"thread_id": "abc123"}}

while True:
  domanda = input("> ")
  if domanda == "stop":
    break
  risposta = agent_executor.invoke({"messages": [HumanMessage(content=domanda)]}, config=config)
  display(Markdown(risposta["messages"][-1].content))
  print("\n\n")



## Analisi del comportamento dell'agente

La cella di codice che segue permette di visualizzare lo stato dell'agente che comprende il flusso dei messaggi scambiati, comprese le risposte del LLM in cui viene chiesta l'invocazione del tool.

In [ ]:
for msg in risposta["messages"]:
  print (msg.pretty_print(),"\n")